In [247]:
import librosa
import pandas as pd
import numpy as np
import pathlib
import os
from pydub import AudioSegment
import csv

# Preprocessing
au 파일 포맷을 wav 파일로 변환한다. 

In [83]:
genres="blues classical country disco hiphop jazz metal pop reggae rock".split()

for g in genres:
    for filename in os.listdir(f'C:/Users/admin/OneDrive - 경희대학교/2020년 1학기 강의자료/데이터분석캡스톤디자인/프로젝트/데이터/gtzan-genre-collection/genres/{g}'):
        songname = f'C:/Users/admin/OneDrive - 경희대학교/2020년 1학기 강의자료/데이터분석캡스톤디자인/프로젝트/데이터/gtzan-genre-collection/genres/{g}/{filename}'
        sound = AudioSegment.from_file(songname, "au")
        sound.export(f'C:/Users/admin/OneDrive - 경희대학교/2020년 1학기 강의자료/데이터분석캡스톤디자인/프로젝트/데이터/gtzan-genre-collection/genres_wav/{g}/'+filename[:-2]+"wav", format="wav")

183896.52945672921

# Preprocessing
데이터의 진폭 범위를 [-1과 1]로 맞춰준다.

In [12]:
def normalize(vector):
    # Subtract the mean, and scale to the interval [-1,1]
    vector_minusmean = vector - vector.mean()
    return vector_minusmean/np.abs(vector_minusmean).max()

# timbral texture feature vector extraction

음색 특징으로 사용할 것은 analysis window에 대한 spectral centroid, rolloff, fulx, zerocrossing의 평균과 분산, mfcc의 앞에서부터 5개의 vector에 대한 평균과 분산, 그리고 low energy로 총 19차원의 feature vector를 뽑는다.

###  사용한 음색 특징에 대한 설명

1. spectral centroid : STFT 연산을 하여 나온 feaquency domain에 대한 주파수의 중심점을 나타낸다. 주파수와 진폭에 대한 가중평균. 어느 중심이 되는 주파수

2. time domain zero crossing rate : time domain에서 x축을 얼마나 많이 지나가는지에 대한 횟루를 나타낸다. -> 시끄러운 정도. 

3. Spectral Rolloff : 낮은 주파수 영역의 신호 에너지 집중도. 무성음과 유성음을 구분하는데, 무성음은 고주파 영역을 많이 차지함. frequency domain에서 우측으로 치중된 그래프일수록(right skewed distribution)일수록 값이 크다. 

4. Spectral Flux : 신호의 스펙트럼이 얼마나 빠르게 변화하는지를 측정하는 양. 이전프레임과 현재 프레임의 스펙트럼의 차이

5. low energy feature : texture window(1s)에 대해 구한 RMS의 평균 보다 작은 RMS를 가진 analysis window(20ms)의 비율. 조용한 분위기에서 말이 많을수록 값이 크고, 연속적인 현악기 음악은 낮음.

6. MFCC : 인간이 듣는 방식으로 음색 체크

In [254]:
header = "filename spectral_centroid_mean spectral_centroid_variance spectral_rolloff_mean spectral_rolloff_variance spectral_flux_mean spectral_flux_variance zero_crossing_rate_mean zero_crossing_rate_variance low_energy "
for i in range(1, 6):
    header+=f'mfcc{i}_mean '
    header+=f'mfcc{i}_variance '
    
header+='label'
header=header.split()

file = open('gtzan_feature.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)

                    
genres="blues classical country disco hiphop jazz metal pop reggae rock".split()

for g in genres:
    for filename in os.listdir(f'C:/Users/admin/OneDrive - 경희대학교/2020년 1학기 강의자료/데이터분석캡스톤디자인/프로젝트/데이터/gtzan-genre-collection/genres_wav/{g}'):
        songname = f'C:/Users/admin/OneDrive - 경희대학교/2020년 1학기 강의자료/데이터분석캡스톤디자인/프로젝트/데이터/gtzan-genre-collection/genres_wav/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        y=normalize(y)
        to_append=""
        centroid = librosa.feature.spectral_centroid(y=y, sr=sr) #spectral centroid
        rolloff=librosa.feature.spectral_rolloff(y, sr) #spectral rolloff
        flux=librosa.onset.onset_strength(y, sr) #spectral flux
        zcr=librosa.feature.zero_crossing_rate(y) #zero crossing rate
        mfcc=librosa.feature.mfcc(y=y, sr=sr)[:5] #first five mfcc vectors
        
        rmse=librosa.feature.rms(y=y, hop_length=sr)
        rmsmean=rmse.mean()
        low_energy=((y<rmsmean).sum())/y.shape[0] #low energy
        
        to_append = f'{filename} {centroid.mean()} {centroid.var()} {rolloff.mean()} {rolloff.var()} {flux.mean()} {flux.var()} {zcr.mean()} {zcr.var()} {low_energy} '    
        for e in mfcc:
            to_append += f'{e.mean()} {e.var()} '
        to_append += f'{g}'
        
        file = open('gtzan_feature.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())